# SnowflakeConnector Testing Notebook

In [1]:
# Import required modules and classes
from config import SnowflakeConfig
import pandas as pd
from utils.snowflake_utils import SnowflakeConnector

In [2]:
# Initialize configuration and connector

config = SnowflakeConfig(
    database="PROJECTX",
    schema="PUBLIC",
    search_service="CC_SEARCH_SERVICE_CS",
    warehouse="COMPUTE_WH",
    stage_name="@docs"
)

connector = SnowflakeConnector(config)
print("Connector initialized successfully")

Connection Parameters (excluding password):
Account: lmb53062.us-east-1
User: STEPHANIENGUYEN
Database: PROJECTX
Schema: PUBLIC
Warehouse: COMPUTE_WH
Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS
Connector initialized successfully


In [3]:
res = connector.get_similar_chunks_search_service(query="Why do people resist switching to alternative opportunities when their current livelihood becomes unsustainable?", num_chunks=3)
res

'{"results":[{"chunk":"informal taxation and incentives for conformity, in the extreme leading to what Acemoglu and Robinson (2019) refer to as the \\\\\\"cage of norms.\\\\\\" An important next step is to incorporate such adverse aspects of community relations and investigate when more favorable employment outcomes support positive community norms and when they instead abet negative or discriminatory norms.Third, our work also points towards new empirical explorations. More work is needed to investigate whether there is a causal relationship between the disappearance of attractive em ployment opportunities for workers and the retrenchment of civic life in local communities, and whether this causal effect works through the mechanisms highlighted by our framework. The evidence presented in, among others, Wilson (1996), Black, McKinnish, and Sanders (2005), and Autor, Dorn, and Hanson (2019) suggests that this is plausible, but the exact causal mech- anisms have yet to be studied empiric

In [19]:
for item in connector.root.databases["PROJECTX"].schemas["PUBLIC"].stages["docs"].list_files():
    print(item)

name='docs/Employment and Community - Socioeconomic Cooperation and Its Breakdown.pdf' size='417598' md5='67c5e9441f90342dec4b80fef92f148d' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Help_for_the_Heartland_20240815.pdf' size='9707566' md5='262dcba9373b9091c94a03f7a9358e69-2' last_modified='Mon, 20 Jan 2025 21:15:53 GMT'
name='docs/Learning from Ricardo and Thompson - Machinery and Labor in the Early Industrial Revolution - and in the Age of AI.pdf' size='346719' md5='dae373a77eaad42296bad5bd0c5a65bd' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Mistrust, Misperception, and Misunderstanding - Imperfect Information and Conflict Dynamics.pdf' size='971967' md5='a5631684bb0377e26c7d2d67f88daa4e' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Tasks at Work - Comparative Advantage, Technology and Labor Demand.pdf' size='5511832' md5='88092466e50d87ccc480ed43e5f2fe9c-2' last_modified='Mon, 20 Jan 2025 21:15:54 GMT'
name='docs/The Simple Macroeconomics of AI

In [ ]:
from utils.snowflake_upload import upload_to_stage


upload_to_stage()

# Test chat

In [ ]:
# #agents initialization
# from assistance.user_proxy import IntentClassifier, UserProxy
# from assistance.web_search_agent import WebSearchAgent
# from assistance.writer_agent import WriterAgent
# from assistance.critics_agent import CriticAgent
# from assistance.documents_reading_agent import DocumentReadingAgent
# from services.search_service import create_prompt, create_web_search_prompt, generate_request_to_recipient, reflection_message


# def search(message: str):
#     #agents initialization
#     user_proxy = UserProxy()
#     web_search_agent = WebSearchAgent()
#     writer_agent = WriterAgent()
#     critic_agent = CriticAgent()
#     document_reading_agent = DocumentReadingAgent()
#     web_search_intent = IntentClassifier()

#     # Sequential chat configuration
#     user_proxy.register_nested_chats(
#         chat_queue= [
#             {
#                 "recipient": critic_agent, 
#                 "clear_history": True,
#                 "message": reflection_message,
#                 "summary_method": "last_msg", 
#                 "max_turns": 1
#             }
#             ],
#         trigger=writer_agent
#     )
    
#     res = document_reading_agent.get_relevant_information(message=message)
#     search_res = res['content'] 
#     print("search_res:", search_res)
#     web_search_prompt = create_web_search_prompt(search_res=search_res, message=message)

#     is_search = web_search_intent.classify(web_search_prompt)
#     print("is_search: ", is_search)
#     is_search = is_search['content']
   

c:\Users\tnguy\hackathon\projectx\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from services.search_service import search


message = "What is NVDIA stock right now?"
res = search(message=message)
res   

Connection Parameters (excluding password):
Account: lmb53062.us-east-1
User: STEPHANIENGUYEN
Database: PROJECTX
Schema: PUBLIC
Warehouse: COMPUTE_WH
Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS
search_result {"results":[{"chunk":"Market|||\\n|1.3.1|Labour rights......|...n/a|n/a|\\n|1.3.2||.64.83|27|\\n||Management Practice|||\\n|1.3.3||.56.51|50|\\n|||||\\n||Technology Adoption|||\\n|||||\\n|1.3.5||.8.00|119|\\n|1.3.6||.9.66|101|\\n||Cloud computing ...|||\\n|1.3.7|||69|\\n|||||\\n|||||\\n|||||\\n|||||\\n|2|AtTact.................................9...||72|\\n|2.1|External Openness ...|.35.98|92|\\n||Attract Business|||\\n|2.1.1|FDI regulatory restrictiveness....|.65.95|57|\\n|||||\\n||Attract People|||\\n|2.1.3|Migrant stock .|.0.52|133|\\n|2.1.4|International students..|.0.99|105|\\n|2.1.5|Brain gain .|.60.32|32|\\n|2.2|Internal Openness....|...63.18|39|\\n||Social Inclusion|||\\n|2.2.1|Tolerance of mino

"1. * * Summary * * : The current stock price for NVIDIA Corporation is delayed. However, key stock metrics include a 52-week range of $58.58 to $153.13.\n\n2. * * Detailed Analysis * * :\n   - * * Stock Price Status * * : As of the latest information, NVIDIA Corporation's stock price is delayed.\n   - * * 52-Week Range * * : The stock has fluctuated between $58.58 and $153.13 over the past 52 weeks.\n\n3. * * Citations * * :\n   - The information provided is derived from the context given, which includes details about NVIDIA Corporation's stock performance."

In [2]:
res[-1].chat_history[-1]['content']

"1. Summary : The strategic competition between the US and China involves economic decoupling, with Vietnam playing a key role by strengthening its economic ties with the US and reducing its reliance on China.\n\n2. Detailed Analysis : The document highlights that Vietnam has strengthened its diplomatic ties with the US, particularly in economic cooperation. This alignment serves the US's strategic interests by providing an alternative market to China. Vietnam has actively sought to diversify its export markets and decrease its dependence on China, which aligns with the US's economic goals in the region. This economic cooperation has made Vietnam a crucial partner for the US in its strategic competition with China.\n\n3. Citations : 1. vietnam-2024-outlook-report-en.pdf"